In [20]:
"""
Faster RCNN Flow
1. Feature extraction from image
2. Creating anchor targets
3. Locations and objectness score prediction from the RPN network
4. Taking the top N locations and their objectness scores aka proposal layer
5. Passing these top N locations through Fast R-CNN network and generating locations and cls predictions for each location is suggested in 4.
6. generating proposal targets for each location suggested in 4
7. Using 2 and 3 to calculate rpn_cls_loss and rpn_reg_loss.
8. using 5 and 6 to calculate roi_cls_loss and roi_reg_loss.
"""

# Flow
# 1. region Proposal network(RPN) -> 3 * 3 sliding window 당 anchor boxes 9개가 나온다.
# 2. RPN loss function
# 3. Region of interest Pooling (ROI)
# 4. ROI loss functions

import torch
import torchvision
import numpy as np
import torch.nn as nn

# 기본 이미지
image = torch.zeros((1,3,800,800)).float()

# y1, x1, y2, x2 format으로 이루어져 있음 / 현재는 2개의 B.B GT 생성.
bbox = torch.FloatTensor([[20, 30, 400, 500], [300, 400, 500, 600]])
# 위에서 2개의 B.B의 object를 명시한다.
labels = torch.LongTensor([6,8])

sub_sample = 16


In [21]:
dummy_img = torch.zeros((1,3, 800, 800)).float()
# print(dummy_img)

model = torchvision.models.vgg16(pretrained=True)
# Q. torchvision cuda 찾아보기
fe = list(model.features)
#print(fe)

In [22]:
img_temp = torch.zeros((1,3, 224, 224)).float()
for i in fe:
    print(i)
    img_temp = i(img_temp)
    print(img_temp.size())
    print("\n")

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
torch.Size([1, 64, 224, 224])


ReLU(inplace=True)
torch.Size([1, 64, 224, 224])


Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
torch.Size([1, 64, 224, 224])


ReLU(inplace=True)
torch.Size([1, 64, 224, 224])


MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
torch.Size([1, 64, 112, 112])


Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
torch.Size([1, 128, 112, 112])


ReLU(inplace=True)
torch.Size([1, 128, 112, 112])


Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
torch.Size([1, 128, 112, 112])


ReLU(inplace=True)
torch.Size([1, 128, 112, 112])


MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
torch.Size([1, 128, 56, 56])


Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
torch.Size([1, 256, 56, 56])


ReLU(inplace=True)
torch.Size([1, 256, 56, 56])


Conv2d(256, 256, kernel_size=(3, 3),

In [23]:
# Layer를 저장하는 공간이다. 
reg_features = []

k = dummy_img.clone() # 깊은복사. copyTo: 얕은 복사 --> 즉 원본이 변하면 output도 변한다.
for i in fe:
    
    # K = feature map을 의미한다. 
    k = i(k)
    
    # VGG network의 마지막 부분만 제외한다.
    # 3 * 3 * 512 까지만 conv을 적용한다.
    # conv5_3 층 까지만 적용하는 것이다.
    # 즉 마지막 층의 maxpooling 전까지 적용 
    if k.size()[2] < 800 // 16:
        break
        
    reg_features.append(i)
    out_channels = k.size()[1]

print(len(reg_features)) # 31
print(out_channels) # 512



30
512


In [24]:
# convert list into a sequential module
faster_rcnn_fe_extractor = nn.Sequential(*reg_features)
print(faster_rcnn_fe_extractor)
print("\n")
out_map = faster_rcnn_fe_extractor(image)
print(out_map.size()) # torch.Size([1, 512, 50, 50])


Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [25]:
# anchor boxes

# 1. Generate Anchor at a feature map location
# 2. Generate Anchor at all the feature map location.
# 3. Assign the labels and location of objects (with respect to the anchor) to each and every anchor.
# 4. Generate Anchor at a feature map location

# 현재는 8, 16, 32 형태로 anchor box를 생성할 것이며
# 비율은 0.5, 1, 2 ratio로 진행할 것이다. 
# sub sampling은 16 ( 800px -> 50px 로 줄였기 때문에 나온 숫자. )
# sub sampling이 무엇을 의미하는 것일 까? 

"""
우리가 anchor를 뽑아 내는 것은 input image에서 뽑아내는 것이다.
즉 현재 input 이 800 * 800 이며 
마지막 feature map의 크기가 50 * 50 이기 때문에 
16 * 16 픽셀이 feature map의 1개의 픽셀과 correspoding 하다는 것이다. 
"""


'\n우리가 anchor를 뽑아 내는 것은 input image에서 뽑아내는 것이다.\n즉 현재 input 이 800 * 800 이며 \n마지막 feature map의 크기가 50 * 50 이기 때문에 \n16 * 16 픽셀이 feature map의 1개의 픽셀과 correspoding 하다는 것이다. \n'

In [33]:
ratio = [0.5, 1, 2]
anchor_scales = [8, 16, 32]

anchor_base = np.zeros((len(ratio) * len(anchor_scales), 4), dtype=np.float32)

print(anchor_base)
print(anchor_base.shape)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
(9, 4)


In [35]:
# 이게 가장 처음의 image 16 * 16 픽셀부분에 대한 anchor box 9개의 좌표를 의미한다.
# 다른 말로 feature map 첫번째 픽셀의 input image에서의 anchor location을 의미한다. 

# 따라서 우리는 17500(50 * 50 * 9) 개의 anchor가 필요하다.

# ctr: sliding window 에 대한 가운데 좌표를 의미한다. 
ctr_y = sub_sample / 2.0
ctr_x = sub_sample / 2.0

for i in range(len(ratio)):
    for j in range(len(anchor_scales)):
        h = sub_sample * anchor_scales[j] * np.sqrt(ratio[i])
        w = sub_sample * anchor_scales[j] * np.sqrt(1.0 / ratio[i])
        print("%d %d %f %f"%(i, j, h, w))
        
        # 0 ~ 9
        index = i * len(anchor_scales) + j
        
        anchor_base[index,0] = ctr_y - h / 2.0
        anchor_base[index,1] = ctr_x - w / 2.0
        anchor_base[index,2] = ctr_y + h / 2.0
        anchor_base[index,3] = ctr_x + w / 2.0
        
print(anchor_base)

0 0 90.509668 181.019336
0 1 181.019336 362.038672
0 2 362.038672 724.077344
1 0 128.000000 128.000000
1 1 256.000000 256.000000
1 2 512.000000 512.000000
2 0 181.019336 90.509668
2 1 362.038672 181.019336
2 2 724.077344 362.038672
[[ -37.254833  -82.50967    53.254833   98.50967 ]
 [ -82.50967  -173.01933    98.50967   189.01933 ]
 [-173.01933  -354.03867   189.01933   370.03867 ]
 [ -56.        -56.         72.         72.      ]
 [-120.       -120.        136.        136.      ]
 [-248.       -248.        264.        264.      ]
 [ -82.50967   -37.254833   98.50967    53.254833]
 [-173.01933   -82.50967   189.01933    98.50967 ]
 [-354.03867  -173.01933   370.03867   189.01933 ]]


In [39]:
fe_size = (800//16)
ctr_x = np.arange(16, (fe_size+1) * 16, 16)
ctr_y = np.arange(16, (fe_size+1) * 16, 16)
print(ctr_x)

[ 16  32  48  64  80  96 112 128 144 160 176 192 208 224 240 256 272 288
 304 320 336 352 368 384 400 416 432 448 464 480 496 512 528 544 560 576
 592 608 624 640 656 672 688 704 720 736 752 768 784 800]


In [46]:
ctr = torch.zeros((len(ctr_x)*len(ctr_y),2)).float()
index = 0
for x in range(len(ctr_x)):
    for y in range(len(ctr_y)):
        ctr[index, 1] = ctr_x[x] - 8.0
        ctr[index, 0] = ctr_y[y] - 8.0
        index = index + 1
print(ctr.shape)
        

torch.Size([2500, 2])


In [51]:
anchors = torch.zeros((fe_size * fe_size * 9),4)

index = 0
for c in ctr:
    # C = ctr의 값들을 불러온다. ex) tensor([8. , 8.])
    ctr_y , ctr_x = c
    for i in range(len(ratio)):
        for j in range(len(anchor_scales)):
            h = sub_sample * anchor_scales[j] * np.sqrt(ratio[i])
            w = sub_sample * anchor_scales[j] * np.sqrt(1./ ratio[i])
            
            anchors[index, 0] = ctr_y - h / 2.
            anchors[index, 1] = ctr_x - w / 2.
            anchors[index, 2] = ctr_y + h / 2.
            anchors[index, 3] = ctr_x + w / 2.
            
            index += 1
print(anchors.shape)

torch.Size([22500, 4])


In [64]:
# image 의 모든 anchor box에 대해서 객체의 레이블과 위치를 할당해야 한다. 
# intersection - over - union이 가장 높은 앵커
# IoU가 GT와 0.7 겹치는 앵커 박스를 찾는다.


In [66]:
# 일단 생성된 anchor box 에서 이미지 안에 있는 것만 선택 
# index를 선택한다. 
index_inside = np.where(
        (anchors[:, 0] >= 0) &
        (anchors[:, 1] >= 0) &
        (anchors[:, 2] <= 800) &
        (anchors[:, 3] <= 800)
    )[0]
print(index_inside.shape)

(8940,)


In [130]:
label = np.empty((len(index_inside), ), dtype=np.int32)
label.fill(-1)
print(label)
print(label.shape)
#Out = (8940, )

[-1 -1 -1 ... -1 -1 -1]
(8940,)


In [79]:
valid_anchor = anchors[index_inside]
print(valid_anchor)

tensor([[ 13.4903,  10.7452, 194.5097, 101.2548],
        [ 29.4903,  10.7452, 210.5097, 101.2548],
        [ 45.4903,  10.7452, 226.5097, 101.2548],
        ...,
        [573.4904, 698.7452, 754.5096, 789.2548],
        [589.4904, 698.7452, 770.5096, 789.2548],
        [605.4904, 698.7452, 786.5096, 789.2548]])


In [ ]:
# 선택된 anchor 중에서 IoU를 계산해서 object가 있는지 없는지를 판단해야 한다. 

In [114]:
# IoU 계산 
ious = np.zeros((len(valid_anchor),2), dtype = np.float32)

# for문 2개 돌리면 속도가 너무 느릴것 같다. 이부분 어떻게 하면 속도를 올릴까?
# numpy 배열 어떻게 적용할지 생각해 보기 
for index_anchor, i in enumerate(valid_anchor):
    y1, x1, y2, x2 = i
    anchor_area = (y2 - y1) * (x2 - x1)
    for index_bb, j in enumerate(bbox):
        b_y1, b_x1, b_y2, b_x2 = j
        box_area = (b_y2 - b_y1) * (b_x2 - b_x1)
        
        # 우리가 알고 싶은 것은 예측한 anchor 박스와 GT가 얼마나 겹치는지 판단.
        # 왼쪽 상단 좌표값은 큰 값으로
        # 오른쪽 하단 좌표값은 작은 값으로 선택 
        inter_x1 = max([b_x1, x1])
        inter_y1 = max([b_y1, y1])
        inter_x2 = min([b_x2, x2])
        inter_y2 = min([b_y2, y2])
        
        if (inter_x1 < inter_x2) and (inter_y1 < inter_y2):
            iter_area = (inter_y2 - inter_y1) * (inter_x2 - inter_x1)
            iou = iter_area / (anchor_area+ box_area - iter_area)            
        else:
            iou = 0.
            
        ious[index_anchor, index_bb] = iou
# shape 8940 * 2
# 이게 의미하는 바는 현재 image 에서는 800 * 800 안에 있는 anchor 수가 8940 이며
# 지금 image 안에는 2개의 object가 존재하고 
# 각각의 object 당 object가 들어있을 경우의 확률 및 object가 하나도 없는 때는 0을 포함하고 있음 
print(ious.shape)
print(ious)

(8940, 2)
[[0.06811669 0.        ]
 [0.07083762 0.        ]
 [0.07083762 0.        ]
 ...
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]]


In [126]:
# X축끼리 비교 Y축 끼리 비교 
gt_argmax_ious = ious.argmax(axis=0)
print(gt_argmax_ious)

print(ious[2262,0])
print(ious[5620,1])
gt_max_ious = ious[gt_argmax_ious, np.arange(ious.shape[1])]
print(gt_max_ious)

[2262 5620]
0.68130493
0.61035156
[0.68130493 0.61035156]


In [107]:
# 2개의 object 대소 비교하는 것이다. 
# 가자큰 object 번호를 선택하게 되어 있다. 
# 따라서 크기는 8940 이며 여기서의 0과 1은 오브젝트를 의미한다. 
argmax_ious = ious.argmax(axis=1)
print(argmax_ious.shape)
print(argmax_ious)

# 해당 object의 확률 값을 의미한다. 
max_ious = ious[np.arange(len(index_inside)), argmax_ious]
print(max_ious)

(8940,)
[0 0 0 ... 0 0 0]
[0.06811669 0.07083762 0.07083762 ... 0.         0.         0.        ]


In [108]:
gt_argmax_ious = np.where(ious == gt_max_ious)[0]
print(gt_argmax_ious)

[2262 2508 5620 5628 5636 5644 5866 5874 5882 5890 6112 6120 6128 6136
 6358 6366 6374 6382]


In [113]:
a = np.arange(8).reshape(4,2) 
print(a)
print(a.argmax(axis = 0))
print(a.argmax(axis = 1))

[[0 1]
 [2 3]
 [4 5]
 [6 7]]
[3 3]
[1 1 1 1]


In [ ]:
"""
1. argmax_ious — Tells which ground truth object has max iou with each anchor.
--> GT의 object 값을 의미한다. 

2. max_ious — Tells the max_iou with ground truth object with each anchor.
--> object의 IoU값을 의미한다. 

3. gt_argmax_ious — Tells the anchors with the highest Intersection-over-Union (IoU) overlap with a ground-truth box.
--> GT와 가장 크게 겹치는 anchor 박스를 의미한다. 
"""

In [158]:
# 해당 부분 paper 에서는 0.7로 설정되어 있다. 
pos_iou_threshold = 0.6
neg_iou_threshold = 0.3

# label 은 8940 의 shape을 가지고 있다. 
# 해당 shape 은 image 안에 있는 anchor의 개수를 의미한다.
# 여기서 label을 하는 것은 anchor가 positive 인지 negative 인지를 알 수있다.
# 현재는 -1로 초기화 되어있다. 

# max_ious는 object와 B.B gt가 얼마나 겹치는지 %를 의미한다. 30% 아래이면 0 
label[max_ious< neg_iou_threshold] = 0
label[gt_argmax_ious] = 1 # IoU가 가장 큰 값도 1
label[max_ious >= pos_iou_threshold] = 1 # 0.7 이상이여야지만 1이다. 


In [159]:
print(np.where(label == 1)[0])

[2262 2269 2276 2508 2515 2522 2754 2761 2768 3000 3007 3959 3966 4233
 4240 5620 5628 5636 5644 5866 5874 5882 5890 6112 6120 6128 6136 6358
 6366 6374 6382]


In [160]:
pos_ratio = 0.5 
n_sample = 256

n_pos = pos_ratio * n_sample
print(n_pos)

128.0


In [164]:
# positie samples
pos_index = np.where(label == 1)[0]
print(len(pos_index))

# 만약 posive 가 많을 경우 해당 부분의 개수를 줄여주는 코드
if len(pos_index) > n_pos:
    disable_index = np.random.choice(pos_index, size = (len(pos_index) - n_pos), replace=False)
    label[disable_index] = -1

31


In [168]:
# negative samples code 
# 만약 negative가 많을 경우 빼주는 코드이다. 

n_neg = n_sample * np.sum(label == 1)
print(n_neg)
neg_index = np.where(label == 0)[0]
print(len(neg_index))

if len(neg_index) > n_neg:
    disable_index = np.random.choice(neg_index, size=(len(neg_index) - n_neg), replace = False)
    label[disable_index] = -1

7936
7690


In [173]:
# 모든 anchor에 대한 GT 생성
print(np.where(argmax_ious == 1)[0])
max_iou_bbox = bbox[argmax_ious]
print(max_iou_bbox.shape)
print(max_iou_bbox[0,:])
print(max_iou_bbox[1891,:])


[1891 1897 1903 ... 8561 8565 8569]
torch.Size([8940, 4])
tensor([ 20.,  30., 400., 500.])
tensor([300., 400., 500., 600.])


In [176]:
# input size 안에 있는 anchor의 
# center 와 height 와 width 를 기록한다.
height = valid_anchor[:, 2] - valid_anchor[:, 0]
width = valid_anchor[:, 3] - valid_anchor[:, 1]
ctr_y = valid_anchor[:, 0] + 0.5 * height
ctr_x = valid_anchor[:, 1] + 0.5 * width

In [177]:
# GT의 center 와 height 와 width를 의미한다. 
base_height = max_iou_bbox[:, 2] - max_iou_bbox[:, 0]
base_width = max_iou_bbox[:, 3] - max_iou_bbox[:, 1]
base_ctr_y = max_iou_bbox[:, 0] + 0.5 * base_height
base_ctr_x = max_iou_bbox[:, 1] + 0.5 * base_width

In [3]:
from google_images_download import google_images_download
from PIL import Image
import ssl
import os

ssl._create_default_https_context = ssl._create_unverified_context


def imageCrawling(keyword, dir):
    response = google_images_download.googleimagesdownload()

    arguments = {"keywords": keyword,
                 "limit": 1500,
                 "print_urls": True,
                 "no_directory": True,
                 'output_directory': dir,
                 "chromedriver": "C:/Users/user/Downloads/chromedriver_win32"
                 }
    paths = response.download(arguments)
    print(paths)


def img_delete():
    img_dir = r"C:/Users/user/Pictures/Saved Pictures"
    for filename in os.listdir(img_dir):
        try:
            with Image.open(img_dir + '/' + filename) as im:
                print("ok")
        except:
            print(img_dir + "/" + filename)
            os.remove(img_dir + "/" + filename)


if __name__ == '__main__':
    imageCrawling('do not wash symbol', "C:/Users/user/Pictures/Saved Pictures/")


ModuleNotFoundError: No module named 'google_images_download'